In [46]:
import pandas as pd
import requests
import time

In [ ]:
# GET NASA DATA
# api docs: https://power.larc.nasa.gov/api/pages/?urls.primaryName=Monthly+%26+Annual#/Data%20Requests/monthly_single_point_data_request_api_temporal_monthly_point_get

counties = pd.read_csv('counties.csv')

BASE_URL = "https://power.larc.nasa.gov/api/temporal/monthly/point"
PARAMETERS = "ALLSKY_SFC_SW_DWN,RH2M,WS2M,WS2M_MAX" # solar radiation, humidity, wind speed avg, wind speed max
START_DATE = "1984"
END_DATE = "2024"

all_data = []
for idx, row in counties.iterrows():
    county_name = row['NAME']
    
    # specify counties by their lon/lat centroids
    lat = row['INTPTLAT']
    lon = row['INTPTLONG']

    print(f"[{idx+1}/{len(counties)}] Getting data for: {county_name}")
    
    params = {
        'start': START_DATE,
        'end': END_DATE,
        'latitude': lat,
        'longitude': lon,
        'community': 'ag',
        'parameters': PARAMETERS,
        'format': 'json'
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        
        data = response.json()
        data = data['properties']['parameter']
        
        yearly_data = {}
        
        for param_name, monthly_values in data.items():
            for month_key, value in monthly_values.items():
                year = int(month_key[:4])
                month = int(month_key[4:6])
                
                if year not in yearly_data:
                    yearly_data[year] = {
                        'County': county_name,
                        'Year': year,
                        'solar_radiation': [],
                        'humidity': [],
                        'wind_speed': [],
                        'wind_speed_max': []
                    }
                
                # use april to september
                if month >= 4 and month <= 9:
                    if param_name == 'ALLSKY_SFC_SW_DWN':
                        yearly_data[year]['solar_radiation'].append(value)
                    elif param_name == 'RH2M':
                        yearly_data[year]['humidity'].append(value)
                    elif param_name == 'WS2M':
                        yearly_data[year]['wind_speed'].append(value)
                    elif param_name == 'WS2M_MAX':
                        yearly_data[year]['wind_speed_max'].append(value)

        for year, data_dict in yearly_data.items():
            
            # calculate aggregates for the growing season
            row_data = {
                'County': county_name,
                'Year': year,
                'solar_radiation': sum(data_dict['solar_radiation']) / len(data_dict['solar_radiation']),
                'humidity': sum(data_dict['humidity']) / len(data_dict['humidity']),
                'wind_speed': sum(data_dict['wind_speed']) / len(data_dict['wind_speed']),
                'wind_speed_max': max(data_dict['wind_speed_max'])
            }

            all_data.append(row_data)

    except Exception as e:
        print(e)

    time.sleep(0.5)

nasa_df = pd.DataFrame(all_data)

nasa_df = nasa_df.sort_values(['County', 'Year']).reset_index(drop=True)
nasa_df.to_csv('nasa_weather.csv', index=False)
nasa_df

[1/72] Getting data for: Adams County
[2/72] Getting data for: Ashland County
[3/72] Getting data for: Barron County
[4/72] Getting data for: Bayfield County
[5/72] Getting data for: Brown County
[6/72] Getting data for: Buffalo County
[7/72] Getting data for: Burnett County
[8/72] Getting data for: Calumet County
[9/72] Getting data for: Chippewa County
[10/72] Getting data for: Clark County
[11/72] Getting data for: Columbia County
[12/72] Getting data for: Crawford County
[13/72] Getting data for: Dane County
[14/72] Getting data for: Dodge County
[15/72] Getting data for: Door County
[16/72] Getting data for: Douglas County
[17/72] Getting data for: Dunn County
[18/72] Getting data for: Eau Claire County
[19/72] Getting data for: Florence County
[20/72] Getting data for: Fond du Lac County
[21/72] Getting data for: Forest County
[22/72] Getting data for: Grant County
[23/72] Getting data for: Green County
[24/72] Getting data for: Green Lake County
[25/72] Getting data for: Iowa Co

,County,Year,solar_radiation,humidity,wind_speed,wind_speed_max
0,Adams County,1984,18.366667,73.363333,3.186667,12.36
1,Adams County,1985,18.860000,72.008333,3.131667,10.48
2,Adams County,1986,17.556667,76.470000,3.145000,10.83
3,Adams County,1987,18.656667,71.866667,2.908333,8.95
4,Adams County,1988,20.330000,58.116667,3.166667,9.76
...,...,...,...,...,...,...
2947,Wood County,2020,19.391667,81.131667,2.900000,8.65
2948,Wood County,2021,19.175000,77.456667,2.635000,7.26
2949,Wood County,2022,18.413333,75.690000,3.045000,10.56
2950,Wood County,2023,18.926667,73.635000,2.733333,10.43


In [ ]:
# GET PRISM DATA
# prism data: https://prism.oregonstate.edu/explorer/bulk.php

csv_list = ['prism_weather_1984-1994.csv', 'prism_weather_1995-2009.csv', 'prism_weather_2010-2024.csv']
dfs = []

for csv in csv_list:
    df = pd.read_csv(csv)

    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # use april to september
    growing_season = df[df['Month'].between(4, 9)]

    aggregated = growing_season.groupby(['Name', 'Year']).agg({
        'ppt (inches)': 'sum',            # total precipitation during growing season
        'tmin (degrees F)': 'mean',       # average minimum temperature
        'tmean (degrees F)': 'mean',      # average mean temperature
        'tmax (degrees F)': 'mean',       # average maximum temperature
        'tdmean (degrees F)': 'mean',     # average dew point temperature
        'vpdmax (hPa)': 'mean',           # average maximum vapor pressure deficit
        'vpdmin (hPa)': 'mean'            # average minimum vapor pressure deficit
    }).reset_index()

    dfs.append(aggregated)

prism_df = pd.concat(dfs)

prism_df = prism_df.rename(columns={
    'Name': 'County',
    'ppt (inches)': 'precip_total',
    'tmin (degrees F)': 'temp_min',
    'tmean (degrees F)': 'temp_mean',
    'tmax (degrees F)': 'temp_max',
    'tdmean (degrees F)': 'dewpoint_mean',
    'vpdmax (hPa)': 'vpd_max',
    'vpdmin (hPa)': 'vpd_min'
})

prism_df = prism_df.sort_values(['County', 'Year']).reset_index(drop=True)
prism_df.to_csv('prism_weather.csv', index=False)

prism_df



,County,Year,precip_total,temp_min,temp_mean,temp_max,dewpoint_mean,vpd_max,vpd_min
0,Adams,1984,22.63,49.066667,61.150000,73.200000,48.966667,16.306667,1.061667
1,Adams,1985,22.58,49.083333,61.533333,74.016667,49.916667,16.850000,0.970000
2,Adams,1986,26.01,50.050000,61.783333,73.466667,51.366667,15.193333,0.750000
3,Adams,1987,19.73,50.766667,63.266667,75.766667,52.150000,17.251667,0.633333
4,Adams,1988,16.91,49.883333,64.000000,78.083333,48.666667,22.423333,1.451667
...,...,...,...,...,...,...,...,...,...
2947,Wood,2020,22.71,49.300000,60.300000,71.316667,49.100000,15.001667,0.976667
2948,Wood,2021,29.45,50.466667,61.933333,73.433333,50.966667,15.835000,1.151667
2949,Wood,2022,22.82,49.700000,60.333333,70.950000,50.266667,14.483333,0.971667
2950,Wood,2023,17.49,49.766667,61.750000,73.700000,50.100000,17.655000,0.930000


In [ ]:
# GET SOIL DATA
# soil grids data: https://rest.isric.org/soilgrids/v2.0/docs#/default/

counties = pd.read_csv('counties.csv')

BASE_URL = "https://rest.isric.org/soilgrids/v2.0/properties/query"
# soil ph (H2O), clay content, sand content, silt content, soil organic carbon, total nitrogen, bulk density, cation exchange capacity 
PROPERTIES = ["phh2o", "clay", "sand", "silt", "soc", "nitrogen", "bdod", "cec"]

# offsets (in degrees; 1-2 km shifts) to try if initial centroid fails
OFFSETS = [(0, 0), (0.01, 0), (-0.01, 0), (0, 0.01), (0, -0.01), (0.02, 0), (-0.02, 0), (0, 0.02), (0, -0.02)]

all_data = []
for idx, row in counties.iterrows():
    county_name = row['NAME']
    
    # original centroid
    original_lat = row['INTPTLAT']
    original_lon = row['INTPTLONG']

    print(f"[{idx+1}/{len(counties)}] Getting data for: {county_name}")
    
    success = False
    for delta_lat, delta_lon in OFFSETS:
        current_lat = original_lat + delta_lat
        current_lon = original_lon + delta_lon
        
        params = {
            'lat': current_lat,
            'lon': current_lon,
            'depth': ['0-5cm'],
            'property': PROPERTIES,
            'value': ['mean']
        }

        layer_data = {
            'County': county_name         
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()
            
            data = response.json()
            layers = data['properties']['layers']

            for layer in layers:
                val = layer['depths'][0]['values']['mean']
                layer_data[layer['name']] = val

            if layer_data['clay'] is not None:
                all_data.append(layer_data)
                break
            
        except Exception as e:
            print(f"  Error on attempt: {e}")
        
        time.sleep(5)
    
    time.sleep(5)

soil_df = pd.DataFrame(all_data)
print(soil_df.isnull().sum())
soil_df.to_csv('soil_data.csv', index=False)
soil_df

[1/72] Getting data for: Adams County
[2/72] Getting data for: Ashland County
[3/72] Getting data for: Barron County
[4/72] Getting data for: Bayfield County
[5/72] Getting data for: Brown County
[6/72] Getting data for: Buffalo County
[7/72] Getting data for: Burnett County
[8/72] Getting data for: Calumet County
[9/72] Getting data for: Chippewa County
[10/72] Getting data for: Clark County
[11/72] Getting data for: Columbia County
[12/72] Getting data for: Crawford County
[13/72] Getting data for: Dane County
[14/72] Getting data for: Dodge County
[15/72] Getting data for: Door County
[16/72] Getting data for: Douglas County
[17/72] Getting data for: Dunn County
[18/72] Getting data for: Eau Claire County
[19/72] Getting data for: Florence County
[20/72] Getting data for: Fond du Lac County
[21/72] Getting data for: Forest County
[22/72] Getting data for: Grant County
[23/72] Getting data for: Green County
[24/72] Getting data for: Green Lake County
[25/72] Getting data for: Iowa Co

,County,bdod,cec,clay,nitrogen,phh2o,sand,silt,soc
0,Adams County,132,163,192,698,60,489,319,375
1,Ashland County,115,249,258,601,51,216,526,605
2,Barron County,133,150,185,516,55,317,498,677
3,Bayfield County,111,266,187,545,50,480,333,668
4,Brown County,134,289,308,781,66,328,364,429
...,...,...,...,...,...,...,...,...,...
67,Waukesha County,136,189,277,912,64,153,570,374
68,Waupaca County,132,169,236,1029,60,411,353,468
69,Waushara County,132,187,158,918,60,555,287,494
70,Winnebago County,128,225,385,876,63,184,430,527


In [ ]:
# CLEAN AND MERGE

# crop yield data: https://quickstats.nass.usda.gov/
yield_df = pd.read_csv('yield.csv')
yield_df = yield_df[yield_df['Year'].between(1984, 2024)]
yield_df = yield_df[yield_df['County'] != 'OTHER COUNTIES']
yield_df['County'] = yield_df['County'].str.lower()
yield_df = yield_df[yield_df['County'] != 'other (combined) counties']
yield_df = yield_df[['Year', 'County', 'Value']]
yield_df = yield_df.rename(columns={'Value': 'Yield'})

nasa_df = pd.read_csv('nasa_weather.csv')
nasa_df['County'] = nasa_df['County'].str.replace(' County', '', regex=False).str.lower()
nasa_df['County'] = nasa_df['County'].str.replace('st. croix', 'st croix', regex=False)

prism_df = pd.read_csv('prism_weather.csv')
prism_df['County'] = prism_df['County'].str.lower()
prism_df['County'] = prism_df['County'].str.replace('st. croix', 'st croix', regex=False)

soil_df = pd.read_csv('soil_data.csv')
soil_df['County'] = soil_df['County'].str.replace(' County', '', regex=False).str.lower()
soil_df['County'] = soil_df['County'].str.replace('st. croix', 'st croix', regex=False)

final_dataset = pd.merge(yield_df, nasa_df, on=['Year', 'County'], how='left')
final_dataset = pd.merge(final_dataset, prism_df, on=['Year', 'County'], how='left')
final_dataset = pd.merge(final_dataset, soil_df, on=['County'], how='left')

print(final_dataset.isnull().sum())
final_dataset.to_csv('final_dataset.csv', index=False)
final_dataset


Year               0
County             0
Yield              0
solar_radiation    0
humidity           0
wind_speed         0
wind_speed_max     0
precip_total       0
temp_min           0
temp_mean          0
temp_max           0
dewpoint_mean      0
vpd_max            0
vpd_min            0
bdod               0
cec                0
clay               0
nitrogen           0
phh2o              0
sand               0
silt               0
soc                0
dtype: int64


,Year,County,Yield,solar_radiation,humidity,wind_speed,wind_speed_max,precip_total,temp_min,temp_mean,...,vpd_max,vpd_min,bdod,cec,clay,nitrogen,phh2o,sand,silt,soc
0,2024,adams,120.1,19.265000,74.825000,2.965000,12.01,29.87,50.850000,62.600000,...,15.786667,0.503333,132,163,192,698,60,489,319,375
1,2024,green lake,168.3,19.510000,75.540000,2.925000,9.89,24.88,52.316667,63.066667,...,14.053333,0.606667,133,174,289,1244,60,192,518,437
2,2024,juneau,141.0,19.270000,74.825000,2.965000,12.01,30.75,51.283333,63.016667,...,16.023333,0.531667,135,174,245,671,59,393,362,472
3,2024,marquette,126.0,19.265000,74.925000,2.855000,11.23,27.02,51.916667,63.450000,...,16.401667,0.681667,126,158,280,1126,55,371,349,512
4,2024,portage,159.5,18.655000,76.908333,2.808333,10.97,33.20,51.416667,62.500000,...,15.405000,0.740000,123,221,167,637,55,544,289,431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,1984,monroe,113.9,18.430000,73.730000,3.041667,11.14,24.99,48.966667,60.750000,...,15.933333,1.096667,128,148,172,700,56,301,527,453
2599,1984,pepin,109.2,18.653333,74.403333,3.036667,10.36,20.87,48.966667,60.983333,...,16.100000,1.010000,127,218,212,593,58,213,575,740
2600,1984,pierce,106.1,18.683333,74.101667,3.108333,10.64,20.37,45.600000,59.550000,...,16.528333,0.303333,135,249,185,509,60,258,557,467
2601,1984,st croix,102.9,18.311667,75.766667,2.863333,9.86,21.39,48.733333,60.233333,...,14.953333,1.205000,133,239,174,578,60,356,471,482
